Paramètres à estimer :
1) hyperparamètres : 
    - M nombre de documents
    - K nombre de topics
    - W taille du vocabulaire
    - Psi (décrit en dessous)
    - N[i] nombre de mots par document, i allant de 1 à M (peut suivre une loi de Poisson de paramètre Psi)
    - alpha : vecteur de dimension K correspondant au paramètre de la loi de Dirichlet (cf. exemple des ficelles, où les ficelles correspondent aux documents et les topics à leur découpe. alpha est alors le paramètre qui donne les proportions moyennes de chaque topic pour chaque document. alpha est le prior de Dirichlet placé sur theta
    - beta : prior de Dirichlet placé sur fi
    - fi : matrice de taille K*W, où fi[i,j] désigne la probabilité d'un mot w[j] d'appartenir au topic z[i]
   
2) paramètres latents (qui dépendent des hyperparamètres) : pour chaque document :
    - theta : proportion exacte des topics dans chaque document (de dimension M*K)
    - z : topics associés à chaque mot (de dimension N[i])

Estimation des paramètres : on cherche à estimer z, theta et fi. On suppose que alpha et beta sont connus, ainsi que M, K, W, Psi et les N[i]
    3 méthodes :
        a) variational Bayes (VB)
        b) expectation propagation
        c) collapse Gibbs sampling

In [32]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import os
os.chdir('C:\\Users\\Samir\\Downloads')

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'C:\\Users\\Samir\\Downloads'

In [1]:
import numpy as np

class LDA_Model(object):
    def __init__(self, K, alpha, beta):
                 #, fi, theta, z, clean_corp, unique_corpus, n_jkw):
        """constructor
        self, M, W, Psi, N, alpha, beta : les paramètres fixes de notre modèle"""
        import numpy as np
        import math
        #Paramètres fixes du modèle
        self.M = 0
        self.K = K
        self.W = 0
        #self.Psi = Psi
        #nombre de mots par document
        self.N = np.zeros(self.M)
        #paramètre de Dirichlet sur theta
        #self.alpha = np.zeros(K)
        self.alpha = alpha
        #paramètre de Dirichlet sur fi
        self.beta = beta
        #theta par le CVB
        self.theta_cvb = np.zeros((self.M, self.K))
        #fi par le CVB
        self.fi_cvb = np.zeros((self.K,self.W))
        #theta par le Gibbs
        self.theta_gibbs = np.zeros((self.M, K))
        #fi par le Gibbs
        self.fi_gibbs = np.zeros((self.K,self.W))
        #z
        self.z = [[] for _ in range(self.M)]
        #on définit aussi un corpus nettoyé vide
        self.clean_corp = []
        #on définit les uniques mots du corpus pour chaque document
        self.unique_corpus = [[] for _ in range(self.M)]
        #le corpus entier en 1 liste
        self.unique_flattened_corpus = []
        #self.n_jkw = [[[] for k in range(self.K)] for j in range(self.M)]
        self.n_jkw = [[{} for k in range(self.K)] for j in range(self.M)]
        #espérances et variances qui seront nécessaires au calcul de fi et theta
        self.loc_esp__n_jk = np.zeros((self.M,self.K))
        self.loc_var__n_jk = np.zeros((self.M,self.K))
        self.loc_esp__n_kxij = np.zeros((self.M,self.W))
        self.loc_var__n_kxij = np.zeros((self.M,self.W))
        self.loc_esp__n_k = np.zeros(self.K)
        self.loc_var__n_k = np.zeros(self.K)
        self.loc_esp__n_j = np.zeros(self.M)

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
from nltk.stem.porter import PorterStemmer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
import glob
import os


#corpus représente le corpus de documents dont on cherche à appliquer la LDA
#on va d'abord nettoyer le corpus
def clean_corpus(self, corpus):
    clean_corp = self.clean_corp
    files = glob.glob(corpus)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    porter = PorterStemmer()
#     snowball = SnowballStemmer('english')
#     wordnet = WordNetLemmatizer()
#on itère sur le corpus pour récupérer tous les fichiers
    rootdir = 'BBC'
    for subdir, dirs, files in os.walk(rootdir):
        for fle in files:
       #on lit tous les fichiers
            with open(subdir+"\\"+fle) as f:
                text = f.read()
                #pour chaque document du corpus, on effectue une tokenisation
                tokenized_text = word_tokenize(text)
                #on supprime la ponctuation
                tokenized_docs_no_punctuation = []
                for token in tokenized_text: 
                    new_token = regex.sub(u'', token)
                    if not new_token == u'':
                        tokenized_docs_no_punctuation.append(new_token)
                #on supprime les "stop words"
                tokenized_docs_no_stopwords = []
                for word in tokenized_docs_no_punctuation:
                    if not word in stopwords.words('english'):
                        tokenized_docs_no_stopwords.append(word)
                #enfin, on concatène les mots ayant des similarités de sens
                preprocessed_docs = []
                for word in tokenized_docs_no_stopwords:
                    preprocessed_docs.append(porter.stem(word))
                #on supprime les mots de moins de 3 caractères
                out_length_words = []
                for word in preprocessed_docs:
                    if(len(word)>3):
                        out_length_words.append(word)
                clean_corp.append(out_length_words)    
        #maintenant que le corpus est créé, on supprime enfin les mots qui apparaissent 
        #moins de 10 fois dans le corpus  
        #on "flatten" le corpus précédent pour n'avoir qu'une liste
        flattened_corpus = [y for x in self.clean_corp for y in x]
        counts = Counter(flattened_corpus)
        new_flattened_corpus = [s for s in flattened_corpus if 10 < counts[s]]
        i1=0
        for i in clean_corp:
            for j in i:
                if j not in new_flattened_corpus:
                    clean_corp[i1].remove(j)    
            i1 += 1
        self.clean_corp = clean_corp

In [3]:
#ici, on va définir le nombre de documents M, le nombre de mots par documents N[i], ainsi que
#la taille du vocabulaire W
def define_hyperparams_M_N_W(self):
    #on définit le nombre de documents M
    self.M = len(self.clean_corp)
    #vecteur représentant le nombre de mots par document
    #self.N = N
    clean_corp = self.clean_corp
    #on définit les uniques mots du corpus par document
    unique_corpus = [[] for _ in range(self.M)]
    #maintenant on va définir la taille du vocabulaire
    #on transforme le corpus en une liste
    flattened_corpus = [y for x in self.clean_corp for y in x]
    unique_flattened_corpus = self.unique_flattened_corpus
    for i in flattened_corpus:
        if i not in unique_flattened_corpus:
            unique_flattened_corpus.append(i)
    W = len(unique_flattened_corpus)
    self.W = W
    self.unique_flattened_corpus = unique_flattened_corpus
    #on définit aussi les mots uniques par document
    i1 = 0
    for i in clean_corp:
        for j in i:
            if j not in unique_corpus[i1]:
                unique_corpus[i1].append(j)
        i1 = i1 + 1
    self.unique_corpus = unique_corpus
    self.W = W

In [4]:
#on définit maintenant alpha, le paramètre de la loi de Dirichlet sur theta
#on suppose K, le nombre de topics, fixé
def define_hyperparams_alpha(self):
    K = self.K
    alpha = np.random.dirichlet(np.ones(K),size=1)[0]
    self.alpha = alpha

In [5]:
#on définit aussi beta, le paramètre de la loi de Dirichlet sur fi
def define_hyperparams_beta(self):
    W = self.W
    beta = np.random.dirichlet(np.ones(W),size=1)[0]
    self.beta = beta

In [6]:
#maintenant, on doit assigner aléatoirement un topic à chaque mot de chaque document
def define_z_init(self):
    K = self.K
    clean_corp = self.clean_corp
    alpha = self.alpha
    z = [[] for _ in range(self.M)]
    for i in range(len(clean_corp)):
        for j in range(len(clean_corp[i])):
            #on génère une loi multinomiale selon les probabilités de Dirichlet de alpha
            #on tire un élément et on associe le topic au mot
            p = np.random.multinomial(1, alpha, size=1)
            z[i].append(np.argmax(p))
    self.z = z

In [7]:
#on compte par mot, le nombre de mots pour chaque topic, pour chaque document
#on obtient une matrice à 3 indices
from collections import Counter
from itertools import compress
def compute_n_jkw(self):
    #unique_corpus = self.unique_corpus
    clean_corp = self.clean_corp
    z = self.z
    M = self.M
    K = self.K
    n_jkw = [[{} for k in range(self.K)] for j in range(self.M)]
    #j document
    for j in range(M):
        #k topic
        for k in range(K):
            #on récupère d'abord les mots pour un document, pour un topic donnée
            fil = [x in [k] for x in test.z[j]]
            words_for_j_and_k = list(compress(clean_corp[j], fil))
            #create a counter and then a dictionary of words in the document for a topic
            dictionary = Counter(words_for_j_and_k)
            n_jkw[j][k].update(dictionary)
    self.n_jkw = n_jkw

In [8]:
#on compute n_jk.-ij
def compute_n_jk_without_ij(self, j, k, w):
    n_jkw = self.n_jkw
    if w in self.n_jkw[j][k]:
        dict_n_jkw = n_jkw[j][k]
        #global dict_without_ij
        dict_without_ij = {key : dict_n_jkw[key] for key in dict_n_jkw if key != w}
        return (sum(dict_without_ij.values()), len(dict_without_ij)+1)
    else:
        return 0

In [9]:
#on compute n_jk.-ij^2 pour avoir l'espérance de la variable au carré
def compute_n_jk_without_ij2(self, j, k, w):
    n_jkw = self.n_jkw
    if w in self.n_jkw[j][k]:
        dict_n_jkw = n_jkw[j][k]
        #global dict_without_ij
        dict_without_ij = {key : dict_n_jkw[key] for key in dict_n_jkw if key != w}
        return (sum([i ** 2 for i in dict_without_ij.values()]), len(dict_without_ij)+1)
    else:
        return 0

In [10]:
#on compute l'espérance de n_jk.-ij par rapport à w
def esperance_n_jk_without_ij(self, j, k, w):
    if w in self.n_jkw[j][k]:
        return compute_n_jk_without_ij(self, j, k, w)[0] / compute_n_jk_without_ij(self, j, k, w)[1]
    else:
        return 0

In [11]:
#on compute l'espérance de n_jk.-ij^2
def esperance_n_jk_without_ij2(self, j, k, w):
    if w in self.n_jkw[j][k]:
        return compute_n_jk_without_ij2(self, j, k, w)[0] / compute_n_jk_without_ij2(self, j, k, w)[1]
    else:
        return 0

In [12]:
#maintenant on compute la variance de n_jk.-ij
def variance_n_jk_without_ij(self, j, k, w):
    if w in self.n_jkw[j][k]:
        return esperance_n_jk_without_ij2(self, j, k, w) - (esperance_n_jk_without_ij(self, j, k, w)*esperance_n_jk_without_ij(self, j, k, w))
    else:
        return 0

In [13]:
#on compute l'espérance de n_.k.-ij par rapport à w et par rapport à j
def esperance_n_k_without_ij(self, j, k, w):
    M = self.M
    s = 0
    for j1 in range(M):
        if w in self.n_jkw[j1][k]:
            s = s + esperance_n_jk_without_ij(self, j1, k, w)
    s = s - esperance_n_jk_without_ij(self, j, k, w)
    s = s / (M - 1)
    return s

In [14]:
#on compute l'espérance de n_.k.-ij^2
def esperance_n_k_without_ij2(self, j, k, w):
    M = self.M
    s = 0
    for j1 in range(M):
        if w in self.n_jkw[j1][k]:
            s = s + esperance_n_jk_without_ij2(self, j1, k, w)
    s = s - esperance_n_jk_without_ij2(self, j, k, w)
    s = s / (M - 1)
    return s

In [15]:
#on compute maintenant la variance de n_.k.-ij^2
def variance_n_k_without_ij(self, j, k, w):
    return esperance_n_k_without_ij2(self, j, k, w) - (esperance_n_k_without_ij(self, j, k, w)**2)

In [16]:
#on compute n_.kx[ij]-ij
def compute_n_kxij_without_ij(self, j, k, w):
    s = 0
    M = self.M
    n_jkw = self.n_jkw
    #on itère selon tous les documents, sauf le j
    for j1 in [x for x in range(M) if x != j]:
        if w in self.n_jkw[j1][k]:
        #on fait la somme de tous les valeurs ayant w comme clé dans le dictionnaire n_jkw[j1][k]
            s = s + n_jkw[j1][k][w]
    return s

In [17]:
#on compute n_.kx[ij]-ij pour avoir l'espérance de la variable au carré
def compute_n_kxij_without_ij2(self, j, k, w):
    s = 0
    M = self.M
    n_jkw = self.n_jkw
    #on itère selon tous les documents, sauf le j
    for j1 in [x for x in range(self.M) if x != j]:
        if w in self.n_jkw[j1][k]:
        #on fait la somme de tous les valeurs ayant w comme clé dans le dictionnaire n_jkw[j1][k]
            s = s + n_jkw[j1][k][w]
    return s

In [18]:
#on compute l'espérance de n_.kx[ij]-ij
def esperance_n_kxij_without_ij(self, j, k, w):
    M = self.M
    return compute_n_kxij_without_ij(self, j, k, w) / (M-1)

In [19]:
#on compute l'espérance de n_.kx[ij]-ij^2
def esperance_n_kxij_without_ij2(self, j, k, w):
    M = self.M
    return compute_n_kxij_without_ij2(self, j, k, w) / (M-1)

In [20]:
#on compute maintenant la variance de n_.kx[ij]-ij^2
def variance_n_kxij_without_ij(self, j, k, w):
    return esperance_n_kxij_without_ij2(self, j, k, w) - (esperance_n_kxij_without_ij(self, j, k, w)**2)

In [173]:
#initialisation des paramètres en utilisant le Collapse Gibbs Sampling
import pandas as pd
import numpy as np
import random as rdm
import time

def collapse_Gibbs_sampling(self, nbiter, alpha, beta, K):
    start_time = time.time()
    alpha = self.alpha
    beta = self.beta
    K = self.K
    M = self.M
    W = self.W
    n_jkw = self.n_jkw
    unique_corpus = self.unique_corpus
    unique_flattened_corpus = self.unique_flattened_corpus
    z = self.z
    #on compute une première fois toutes les espérances requises
    loc_esp__n_jk = np.zeros((M,K))
    loc_var__n_jk = np.zeros((M,K))
    loc_esp__n_kxij = np.zeros((K,W))
    loc_var__n_kxij = np.zeros((K,W))
    loc_esp__n_k = np.zeros(K)
    loc_var__n_k = np.zeros(K)
    #on compute aussi gamma
    gamma = [[np.zeros(K) for w in range(len(unique_corpus[j]))] for j in range(M)]
    #gamma = [[np.zeros(K) for w in range(len(unique_flattened_corpus))] for j in range(M)]
    #gamma = np.zeros(((M,W,K)))
    
    print("début de computation des espérances et variances")
    #on définit le gamma initial aléatoirement
    for j in range(M):
        #for w in unique_flattened_corpus:
        for w in unique_corpus[j]:
            for k in range(K):
                index_w = unique_corpus[j].index(w)
                gamma[j][index_w][k] = np.random.uniform(0, 1)
    
    for j in range(M):
        for w in unique_corpus[j]:
            index_w = unique_corpus[j].index(w)
            gammasum1 = sum(gamma[j][index_w][:])
            for k in range(K):
                gamma[j][index_w][k] = gamma[j][index_w][k]/gammasum1
                
    #on compute les espérances
    for j in range(M):
        for k in range(K):
            loc_esp__n_jk[j][k] = sum(gamma[j][:][k])
            loc_var__n_jk[j][k] = sum(gamma[j][:][k]*(1 - gamma[j][:][k]))
     
    for k in range(K):
        for j in range(M):
            for w in unique_corpus[j]:
                index_w = unique_corpus[j].index(w)
                loc_esp__n_kxij[k][index_w] += gamma[j][index_w][k]
                loc_var__n_kxij[k][index_w] += gamma[j][index_w][k]*(1 - gamma[j][index_w][k])
            
    for k in range(K):
        loc_esp__n_k[k] = sum(loc_esp__n_jk[:][k])
        loc_var__n_k[k] = sum(loc_esp__n_jk[:][k]*(1 - loc_esp__n_jk[:][k]))
        
                
    print("fin de computation des espérances et variances")          
    for iteration in range(nbiter):
        print("Debut de l'itération ",iteration + 1," : %s secondes ---" % (time.time() - start_time))
        for j in range(M):
            for w in unique_corpus[j]:
                index_w = unique_corpus[j].index(w)
                index_w_unique = unique_flattened_corpus.index(w)
                for k in range(K):
                    p = np.zeros(K)
                    k_topic = z[j][index_w]
                    loc_esp__n_jk[j][k] -= gamma[j][index_w][k]
                    loc_var__n_jk[j][k] -= gamma[j][index_w][k]*(1 - gamma[j][index_w][k])
                    print(loc_esp__n_jk[j][k])
                    
                    loc_esp__n_kxij[k][index_w] -= gamma[j][index_w][k]
                    loc_var__n_kxij[k][index_w] -= gamma[j][index_w][k]*(1 - gamma[j][index_w][k])
                    #print(loc_esp__n_kxij[k][index_w])
                    
                    loc_esp__n_k[k] -= gamma[j][index_w][k]
                    loc_var__n_k[k] -= gamma[j][index_w][k]*(1 - gamma[j][index_w][k])
                    #print(loc_esp__n_k)
                        
                    root = ((0.1 + loc_esp__n_jk[j][k])*(0.1 + loc_esp__n_kxij[k][index_w])/(0.1*W + loc_esp__n_k[k]))
                    a1 = (loc_var__n_jk[j][k])/(2*(0.1 + loc_esp__n_jk[j][k])**2)
                    a2 = (loc_var__n_kxij[k][index_w])/(2*(0.1 + loc_esp__n_kxij[k][index_w])**2)
                    a3 = (loc_var__n_k[k])/(2*(W*0.1 + loc_esp__n_k[k]))
                    #exp = np.exp(-a1 -a2 +a3)  
                    gamma[j][index_w][k] = root
                    #*exp
                    
                #on va normaliser gamma[j][index_w]    
                gammasum = sum(gamma[j][index_w][:])
                for k in range(K):
                    gamma[j][index_w][k] = gamma[j][index_w][k]/gammasum
                    #print(gamma[j][index_w][k])
                            
                #on normalise les probabilités calculées ci-dessus
                p = np.zeros(K)
                u = np.random.uniform(0,1)
                for k1 in range(K-1):
                #print((top[j][1] + alpha[1])* (freq[index_w_unique_flattened_corpus][1] + beta[index_w_unique_flattened_corpus])/(freqTotale[1]*beta[index_w_unique_flattened_corpus]*W))
                    p[k1+1] = p[k1] + gamma[j][index_w][k1]
                for k1 in range(K-1):
                    t_new = 0
                    if u<=p[k1+1]/sum(p):
                        t_new = k1
                        break
                new_topic = t_new
                #print(new_topic)
                z[j][index_w] = new_topic
                    
                for k in range(K):
                    #on actualise ensuite espérances et variances
                    loc_esp__n_jk[j][k] += gamma[j][index_w][k]
                    loc_var__n_jk[j][k] += gamma[j][index_w][k]*(1 - gamma[j][index_w][k])
                    
                    loc_esp__n_kxij[k][index_w] += gamma[j][index_w][k]
                    loc_var__n_kxij[k][index_w] -= gamma[j][index_w][k]*(1 - gamma[j][index_w][k])
                    
                    loc_esp__n_k[k] += gamma[j][index_w][k]
                    loc_var__n_k[k] += gamma[j][index_w][k]*(1 - gamma[j][index_w][k])
    
    #le z calculé, ainsi que les variances et espérances déterminés par CVB pour le calcul de fi et theta
    self.z = z
    self.loc_esp__n_jk = loc_esp__n_jk
    self.loc_var__n_jk = loc_var__n_jk
    self.loc_esp__n_kxij = loc_esp__n_kxij
    self.loc_var__n_kxij = loc_var__n_kxij
    self.loc_esp__n_k = loc_esp__n_k
    self.loc_var__n_k = loc_var__n_k
    
        

In [92]:
#on compute l'espérance de nj.., nécessaire au calcul de theta
def esperance_n_j(self, j):
    M = self.M
    loc_esp__n_jk = self.loc_esp__n_jk
    loc_esp__n_j = self.loc_esp__n_j
    return sum(loc_esp__n_jk[j][:])  

In [100]:
#maintenant on compute theta et fi
def compute_theta_cvb(self):
    alpha = self.alpha
    K = self.K
    M = self.M
    loc_esp__n_jk = self.loc_esp__n_jk
    
    theta_cvb = np.zeros((self.M, K))
    for j in range(M):
        for k in range(K):
            #theta[j,k] = (alpha[k] + esperance_n_jk(self, j, k)) / (K*alpha[k] + esperance_n_j(self, j))
            theta_cvb[j,k] = (0.1 + loc_esp__n_jk[j][k]) / (K*0.1 + esperance_n_j(self, j))
    self.theta_cvb = theta_cvb
            
def compute_fi_cvb(self):
    beta = self.beta
    W = self.W
    K = self.K
    unique_corpus = self.unique_corpus
    unique_flattened_corpus = self.unique_flattened_corpus
    loc_esp__n_kxij = self.loc_esp__n_kxij
    loc_esp__n_k = self.loc_esp__n_k
    
    fi_cvb = np.zeros((K,self.W))
    for k in range(K):
        for w1 in range(len(unique_flattened_corpus)):
            w = unique_flattened_corpus[w1]
            fi_cvb[k,w1] = (0.1 + loc_esp__n_kxij[k][w1]) / (W*0.1 + loc_esp__n_k[k])
    self.fi_cvb = fi_cvb

In [26]:
#on fait le test de l'algorithme
test = LDA_Model(K = 5, alpha =0.1, beta = 0.1)

In [27]:
clean_corpus(test, "corpus_test/*.txt")

In [174]:
define_hyperparams_M_N_W(test)

In [175]:
define_hyperparams_alpha(test)

In [176]:
define_hyperparams_beta(test)

In [177]:
define_z_init(test)

In [178]:
compute_n_jkw(test)

In [179]:
collapse_Gibbs_sampling(test, 50, test.alpha, test.beta, test.K)

début de computation des espérances et variances
fin de computation des espérances et variances
Debut de l'itération  1  : 0.3580000400543213 secondes ---
0.755359939833
0.679170852041
0.642076407488
0.99373132539
0.929661475249
0.708233931659
0.547867626677
0.752424852617
1.04595845698
0.945515132065
0.702504289705
0.579812233745
0.672430896122
0.981858149007
1.06339443142
0.755186133173
0.420397723283
0.664551889053
1.09168280109
1.0681814534
0.878388466227
0.520065610052
0.484461416208
1.16705253876
0.950031968753
0.911254312426
0.618062265555
0.33695026353
1.28839176224
0.845341396248
0.629468941018
0.679394597791
0.427617167437
1.20660455992
1.05691473383
0.617724723218
0.640937387294
0.279236325299
1.36246344187
1.09963812232
0.741232998441
0.485955996556
0.132871339346
1.52603510505
1.1139045606
0.626653862205
0.326077512755
0.0163750377627
1.73539118499
1.29550240229
0.522144331525
0.289262723545
-0.269404571098
2.03716877224
1.42082874378
0.481372886241
0.335005513981
-0.66690

KeyboardInterrupt: 

In [120]:
test.fi_cvb

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [118]:
compute_theta_cvb(test)

In [115]:
compute_fi_cvb(test)

In [117]:
test.fi_cvb

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [129]:
import pandas as pd
import numpy as np
import random as rdm
import time

# docword_KOS = pd.read_csv('docword.kos.txt', sep=" ")
# docword_NIPS= pd.read_csv('docword.nips.txt', sep=" ")
# vocab_KOS= pd.read_csv('vocab.kos.txt', sep=" ")
# vocab_NIPS= pd.read_csv('vocab.nips.txt', sep=" ")

def Gibbs_Sample(self, nbiter, alpha, beta, K):
    start_time = time.time()
    #alpha et beta
    alpha = self.alpha
    beta = self.beta
    #nombre de topics
    K = self.K
    #nombre de documents
    M = self.M
    #V taille du vocabulaire
    W = self.W
    #n_jkw
    n_jkw = self.n_jkw
    #corpus
    unique_corpus = self.unique_corpus
    unique_flattened_corpus = self.unique_flattened_corpus
    #freq matrice de taille W*K = frequence d'apparition dans une classe
    freq = np.zeros((W,K))
    #freqTotale vecteur de taille K = nombre de mots pour chaque classe
    freqTotale= np.zeros(K)
    #top matrice de taille M*K = nombre de mots pour chaque classe, par document
    top = np.zeros((M,K))
    #sumtop vecteur de taille M = somme des mots associés à chaque classe 
    sumtop = np.zeros(M)
    print("Definition des variables terminée : %s secondes ---" % (time.time() - start_time))
#Initialisation des variables
    #initialisation de z
    z = self.z
    #initialisation de top
    for j in range(M):
        for k in range(K):
            for w in unique_corpus[j]:
                if w in n_jkw[j][k]:
                    top[j][k] += n_jkw[j][k][w]
    #intitialisation de freq
    for w in unique_flattened_corpus:
        index_w = unique_flattened_corpus.index(w)
        for k in range(K):
            for j in range(M):
                if w in n_jkw[j][k]:
                    freq[index_w][k] += n_jkw[j][k][w]
    #initialisation de freqTotal
    for k in range(K):
        freqTotale[k] = np.sum(freq[:,k])
    #initialisation de sumtop
    for j in range(M):
        sumtop[j] = np.sum(top[j,:])
    print("Initialisation des variables terminée : %s secondes ---" % (time.time() - start_time))
#Algorithme
    for iteration in range(nbiter):
        print("Debut de l'itération ",iteration + 1," : %s secondes ---" % (time.time() - start_time))
        for j in range(M):
            for w in unique_corpus[j]:
                index_w_unique_corpus = unique_corpus[j].index(w)
                index_w_unique_flattened_corpus = unique_flattened_corpus.index(w)
                t = z[j][index_w_unique_corpus]
                top[j][t] -= 1
                freq[index_w_unique_flattened_corpus][t] -= 1
                freqTotale[t] -= 1
                sumtop[j] -= 1
                u = np.random.uniform(0,1)
                p=np.zeros(K)
                for k in range(K-1):
                    #print((top[j][1] + alpha[1])* (freq[index_w_unique_flattened_corpus][1] + beta[index_w_unique_flattened_corpus])/(freqTotale[1]*beta[index_w_unique_flattened_corpus]*W))
                    p[k+1] = p[k] + (top[j][k] + alpha[k])* (freq[index_w_unique_flattened_corpus][k] + beta[index_w_unique_flattened_corpus])/(freqTotale[k]*beta[index_w_unique_flattened_corpus]*W)
                for k in range(K-1):
                    if u<=p[k+1]/sum(p):
                        t_new = k
                        break
                z[j][index_w_unique_corpus] = t_new
                top[j][t_new] += 1
                freq[index_w_unique_flattened_corpus][t_new] += 1
                freqTotale[t_new] += 1
                sumtop[j] += 1
    print("Fin de l'algorithme %s secondes ---" % (time.time() - start_time))
    print("Computation de theta et fi à %s secondes ---" % (time.time() - start_time))
    #initialisations
    self.fi_gibbs = np.zeros((K,W))
    self.theta_gibbs = np.zeros((M,K))
    for k in range(K):
        for i in range(W):
            self.fi_gibbs[k,i] = (freq[i][k] + beta[i]) / (freqTotale[k] + beta[i]*W)
    for j in range(M):
        for k in range(K):
            self.theta_gibbs[j,k] = (top[j][k] + alpha[k]) / (sumtop[j] + alpha[k]*K)
    #self.fi_gibbs = fi_gibbs
    #self.theta_gibbs = theta_gibbs
    print("fin de computation de theta et fi à %s secondes ---" % (time.time() - start_time))

In [86]:
Gibbs_Sample(test, 100, test.alpha, test.beta, test.K)

Definition des variables terminée : 0.0 secondes ---
Initialisation des variables terminée : 10.14318561553955 secondes ---
Debut de l'itération  1  : 10.14318561553955 secondes ---
Debut de l'itération  2  : 20.107415199279785 secondes ---
Debut de l'itération  3  : 30.28615379333496 secondes ---
Debut de l'itération  4  : 40.49534726142883 secondes ---
Debut de l'itération  5  : 50.129971981048584 secondes ---
Debut de l'itération  6  : 59.66580319404602 secondes ---
Debut de l'itération  7  : 69.24196743965149 secondes ---
Debut de l'itération  8  : 78.43013262748718 secondes ---
Debut de l'itération  9  : 87.90356659889221 secondes ---
Debut de l'itération  10  : 97.32722520828247 secondes ---
Debut de l'itération  11  : 106.83059501647949 secondes ---
Debut de l'itération  12  : 116.04905939102173 secondes ---
Debut de l'itération  13  : 125.4525043964386 secondes ---
Debut de l'itération  14  : 134.9905776977539 secondes ---
Debut de l'itération  15  : 144.529705286026 secondes -

In [90]:
#probabilité pour un mot d'appartenir à un topic
test.fi_gibbs

array([[  1.43998323e-03,   7.17878582e-04,   3.95255071e-03, ...,
          1.42721012e-09,   8.54668901e-10,   4.17194177e-09],
       [  3.36462683e-03,   3.59144288e-04,   3.59140407e-03, ...,
          3.78102000e-05,   3.82566917e-09,   1.89193985e-05],
       [  1.22200696e-03,   1.52717570e-03,   4.88681385e-03, ...,
          1.03208031e-07,   6.18178136e-08,   3.01392086e-07],
       [  2.78976432e-03,   1.88076298e-04,   4.57635581e-03, ...,
          1.05936625e-08,   6.26963162e-05,   3.09640341e-08],
       [  2.91222708e-03,   5.25150778e-04,   4.75018382e-03, ...,
         -2.38618596e-05,  -2.38654886e-05,   2.35809152e-08]])

In [91]:
test.theta_gibbs

array([[  7.25715995e-01,   1.05607338e-01,   5.95138669e-03,
          5.91062132e-02,   1.05540493e-01],
       [  6.92240978e-01,   1.03230699e-01,   1.14941097e-02,
          8.59917108e-02,   1.08793730e-01],
       [  6.28785498e-01,   2.04660472e-01,   1.91654703e-04,
          7.98565450e-02,   8.80417455e-02],
       ..., 
       [  6.37644102e-01,   1.28311049e-01,   1.32952404e-02,
          9.07922208e-02,   1.30491303e-01],
       [  6.16342537e-01,   1.26446591e-01,   5.84943389e-03,
          1.20535860e-01,   1.32057282e-01],
       [  3.86016702e-01,   2.55291200e-01,   1.03380087e-02,
          1.65984644e-01,   1.82311817e-01]])

In [92]:
#proportion des topics dans chaque document
pro = []
for i in range(len(test.theta_gibbs)):
    pro.append(np.argmax(test.theta_gibbs[i]))

In [93]:
max(pro)

0